<a href="https://colab.research.google.com/github/snowgecko/datathon_team_04/blob/main/filterdata_VACs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Imports all the packages.
import numpy as np
import pandas as pd
import os
import pdb
import array

from IPython.core.display import display, HTML
import scipy.stats as stats
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as dates

!pip install scikit-learn
from sklearn import datasets, linear_model
import matplotlib
from sklearn.linear_model import LinearRegression

from google.colab import auth
from google.cloud import bigquery
from google.colab import files

%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# get the amsterdamumcdb package from PyPI repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb


###################SET PROJECT DEFAULTS##################
PROJECT_ID = "dynamic-reef-423509-d1" #@param {type:"string"}

# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'version1_5_0' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = "dynamic-reef-423509-d1"

auth.authenticate_user()
print('Authenticated')

config_gbq = {'query':
          {'defaultDataset': {
              "datasetId": DATASET_ID,
              "projectId": DATASET_PROJECT_ID
              },
           'Location': LOCATION}
           }

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 30000

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset="amsterdamumcdb" + "." + "version1_5_0")
# sets default client settings by re-using the previously defined config
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.6/528.6 kB 6.7 MB/s eta 0:00:00
Authenticated


In [2]:
from google.colab import drive
drive.mount('/content/drive')
#need to select all otherwise it errors out.

# path to base folder (right click on the directory: Datathon, and select 'Add to my Drive' in order to appear on My Drive)
drive_folder = '/content/drive/My Drive/Datathon/'


Mounted at /content/drive


In [3]:
#Shared functions
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=DATASET_PROJECT_ID,
      dialect='standard')


In [99]:
###########################original imported file was ######################################
#3025408 -- o2 conc
#3020716, --fio2
#rslt_df_3d_fi02_2d_onwards = pd.DataFrame()
#rslt_df_3d = pd.read_csv(drive_folder + "rslt_df_3d.csv") # read the dataset to df_data

#############################################################################################
## 1] filtered to those visit_occurrence_ids with fi02 or O2 conc.
## 2] then that are in ICU for 4 days
## 3] then getting the data for 2 calendar days after the Min_Date (something wierd going on with vo_id=0)

rslt_df_3d_fi02_2d_onwards = pd.read_csv(drive_folder + "rslt_df_3d_fi02_2d_onwards.csv") # read the dataset to df_data

##changing the following 4 fields to datetime format
rslt_df_3d_fi02_2d_onwards['measurement_datetime'] = pd.to_datetime(rslt_df_3d_fi02_2d_onwards['measurement_datetime'])
rslt_df_3d_fi02_2d_onwards['measurement_date'] = pd.to_datetime(rslt_df_3d_fi02_2d_onwards['measurement_date'])
rslt_df_3d_fi02_2d_onwards['date_min'] = pd.to_datetime(rslt_df_3d_fi02_2d_onwards['date_min'])
rslt_df_3d_fi02_2d_onwards['date_max'] = pd.to_datetime(rslt_df_3d_fi02_2d_onwards['date_max'])

##############check
rslt_df_3d_fi02_idata.dtypes

##############if I want to save out to csv and individual occurrence_id
#rslt_df_3d_fi02_idata = rslt_df_3d_fi02_2d_onwards[rslt_df_3d_fi02_2d_onwards['visit_occurrence_id'] == 344]
#rslt_df_3d_fi02_idata.to_csv(drive_folder + "rslt_df_3d_fi02_idata.csv", sep=',', index=False, encoding='utf-8')

measurement_id                                 int64
person_id                                      int64
measurement_concept_id                         int64
measurement_date                      datetime64[ns]
measurement_datetime             datetime64[ns, UTC]
measurement_time                              object
measurement_type_concept_id                    int64
operator_concept_id                          float64
value_as_number                              float64
value_as_concept_id                          float64
unit_concept_id                              float64
range_low                                    float64
range_high                                   float64
provider_id                                    int64
visit_occurrence_id                            int64
visit_detail_id                              float64
measurement_source_value                      object
measurement_source_concept_id                float64
unit_source_value                             

In [118]:

# function to label VAC or NO VAC for a given patient data - grouped version
##https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
def is_VAC(df_, _occurrenceid, a_results):
  ##store
  ##df_ is the original rslt_df_3d_fi02_2d_onwards
  rslt_df_3d_fi02_idata = df_.loc[df_['visit_occurrence_id'] == _occurrenceid]

  ###a sub dataframe just to get the min_Fi02 value
  df_min_fi02 = rslt_df_3d_fi02_idata[rslt_df_3d_fi02_idata['value_source_value'] >= 21]
  min_fi02 = df_min_fi02['value_source_value'].min()
  b_delta_unst = False

  start_unst_datetime  = ""
  uns_len = 0
  print("---------------" + str(_occurrenceid) + "---------------")
  #print ("min_fi02 " + str(min_fi02))
  # loop through the rows using iterrows()
  ###########need to iterate through the original passsed dataFrame as making changes############
  for index, row in df_.loc[df_['visit_occurrence_id'] == _occurrenceid].iterrows():
    current_fi02 = df_.loc[index, 'value_source_value']
    #print("current_fi02=" + str(current_fi02))
    delta_min_fi02 = (current_fi02 - min_fi02)
    ###pop new column
    df_.loc[index, 'delta_min_fi02'] = delta_min_fi02
    if (delta_min_fi02 >= 20):
      ##print ("delta_min_fi02=" + str(delta_min_fi02) + " ")
      unst_datetime = df_.loc[index, 'measurement_datetime']
      if (start_unst_datetime == ""):
        start_unst_datetime = unst_datetime
      delta_unst_dt = unst_datetime - start_unst_datetime
      delta_unst_days = delta_unst_dt.days

      df_.loc[index, 'delta_unst_dt'] = delta_unst_dt
      #print("delta_unst_dt=" + str(delta_unst_dt) + " ")
      df_.loc[index, 'delta_unst_days'] = delta_unst_days
      #print("delta_unst_days=" + str(delta_unst_days) + " ")

      if (delta_unst_days >= 2):
        b_delta_unst = True
      #try:
      #except:
      #  print("An exception occurred")
      uns_len += 1
      #rslt_df_3d_fi02_idata['uns_len'] = uns_len
    else:
      start_unst_datetime = ""
    prev_fi02 = current_fi02
  if (b_delta_unst == True):
    print("---------------UNSTABLE-" + str(_occurrenceid) + "---------------")
    a_results.append(str(_occurrenceid))
  df_.loc[df_['visit_occurrence_id'] == _occurrenceid].to_csv(drive_folder + "/idata/rslt_df_3d_fi02_2d_" + str(_occurrenceid) +  ".csv", sep=',', index=False, encoding='utf-8')
  #return a_results


In [ ]:

# function to label VAC or NO VAC for a given patient data - grouped version
##https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
def is_peep_VAC(df_, _occurrenceid, a_results):
  ##store
  ##df_ is the original rslt_df_3d_fi02_2d_onwards
  rslt_df_3d_fi02_idata = df_.loc[df_['visit_occurrence_id'] == _occurrenceid]

  ###a sub dataframe just to get the min_Fi02 value
  df_min_fi02 = rslt_df_3d_fi02_idata[rslt_df_3d_fi02_idata['value_source_value'] >= 21]
  min_fi02 = df_min_fi02['value_source_value'].min()
  b_delta_unst = False

  start_unst_datetime  = ""
  uns_len = 0
  print("---------------" + str(_occurrenceid) + "---------------")
  #print ("min_fi02 " + str(min_fi02))
  # loop through the rows using iterrows()
  ###########need to iterate through the original passsed dataFrame as making changes############
  for index, row in df_.loc[df_['visit_occurrence_id'] == _occurrenceid].iterrows():
    current_fi02 = df_.loc[index, 'value_source_value']
    #print("current_fi02=" + str(current_fi02))
    delta_min_fi02 = (current_fi02 - min_fi02)
    ###pop new column
    df_.loc[index, 'delta_min_fi02'] = delta_min_fi02
    if (delta_min_fi02 >= 20):
      ##print ("delta_min_fi02=" + str(delta_min_fi02) + " ")
      unst_datetime = df_.loc[index, 'measurement_datetime']
      if (start_unst_datetime == ""):
        start_unst_datetime = unst_datetime
      delta_unst_dt = unst_datetime - start_unst_datetime
      delta_unst_days = delta_unst_dt.days

      df_.loc[index, 'delta_unst_dt'] = delta_unst_dt
      #print("delta_unst_dt=" + str(delta_unst_dt) + " ")
      df_.loc[index, 'delta_unst_days'] = delta_unst_days
      #print("delta_unst_days=" + str(delta_unst_days) + " ")

      if (delta_unst_days >= 2):
        b_delta_unst = True
      #try:
      #except:
      #  print("An exception occurred")
      uns_len += 1
      #rslt_df_3d_fi02_idata['uns_len'] = uns_len
    else:
      start_unst_datetime = ""
    prev_fi02 = current_fi02
  if (b_delta_unst == True):
    print("---------------UNSTABLE-" + str(_occurrenceid) + "---------------")
    a_results.append(str(_occurrenceid))
  df_.loc[df_['visit_occurrence_id'] == _occurrenceid].to_csv(drive_folder + "/idata/rslt_df_3d_fi02_2d_" + str(_occurrenceid) +  ".csv", sep=',', index=False, encoding='utf-8')
  #return a_results


In [ ]:
# define a dictionary to assign results and input parameters
#df_results = {'patientunitstayid': [], 'delta_fio2':[], 'delta_peep':[], 'unstable_start': [], 'unstable_end': [], 'VAC_outcome': [], 'IsStable':[]}
###create empty array for the unstable id's to go into.
a_results = []


In [125]:
unique_array_3d = rslt_df_3d_fi02_2d_onwards['visit_occurrence_id'].unique()

print(unique_array_3d.size)
x_it = 0

for x_occurrenceid in unique_array_3d:
  print ("x_occurrenceid=" + str(x_occurrenceid))
  #if x_it >= 1000:
  #  break
  #elif
  if x_it < 1000:
    print ("already done")
  else:
    is_VAC(rslt_df_3d_fi02_2d_onwards, x_occurrenceid, a_results)
  x_it += 1

##iterate over the array and call the function below to check for instability
###individual data (idata) for specific id
#rslt_df_3d_fi02_idata = rslt_df_3d_fi02_2d_onwards[rslt_df_3d_fi02_2d_onwards['visit_occurrence_id'] == 344]
###call function with unique dataframe for that occurrence_id
#is_VAC(rslt_df_3d_fi02_idata)
#rslt_df_3d_fi02_2d_onwards.head(200)
#a_results += a_results

#a_results = 50,66
#['105', '116']
#['182', '188', '189', '190', '240', '278', '279', '285', '299', '302', '332', '366', '375', '407', '411', '412', '439', '442', '503', '569', '625', '654', '700', '707', '800', '813', '859', '877', '903', '919', '943', '966', '1066', '1071', '1115', '1121', '1133', '1140', '1246', '1303', '1327', '1336', '1342', '1353', '1367', '1372', '1411', '1462', '1548', '1568', '1619', '1661', '1677', '1679', '1704', '1729', '1738', '1742', '1756', '1789', '1825', '1860', '1886', '2041', '2125', '2142', '2146', '2148', '2167', '2201', '2264', '2358', '2457', '2482', '2584', '2598', '2600', '2750', '2755', '2762', '2774', '2815', '2831', '2859', '2863', '2905', '2954', '2980', '3010', '3076', '3111', '3124', '3170', '3180', '3260', '3325', '3326', '3334', '3441', '3464', '3586', '3591', '3611', '3617', '3630', '3635', '3784', '3821', '3840', '3888', '3892', '3909', '3910', '3943', '3952', '3973', '3974', '4001', '4038', '4067', '4068', '4194', '4216', '4221', '4332', '4338', '4357', '4390', '4405', '4409', '4500', '4557', '4561', '4651', '4764', '4781', '4788', '4839', '4908', '4993', '5008', '5011', '5108', '5113', '5214', '5235', '5241', '5297', '5363', '5387', '5401', '5503', '5515', '5547', '5592', '5605', '5636', '5651', '5725', '5817', '5828', '5929', '5977', '5986', '6014', '6021', '6037', '6056', '6059', '6107', '6122', '6134', '6144', '6145', '6153', '6178', '6182', '6184', '6203', '6219', '6223', '6233', '6238', '6266', '6283', '6293', '6355', '6370', '6454', '6457', '6463', '6484', '6496', '6499', '6500', '6555', '6562', '6594', '6635', '6640', '6674', '6681', '6682', '6710', '6724', '6743', '6762', '6771', '6820', '6826', '6829', '6903', '6919', '6936', '6945', '7035', '7040', '7091', '7103', '7114', '7126', '7163', '7215', '7227', '7262', '7306', '7345', '7393', '7431', '7442', '7447', '7576', '7599', '7619', '7669', '7761', '7772', '7789', '7792', '7796', '7827', '7839', '7876', '7885', '7889', '7897', '7899', '7929', '7954', '7965', '7966', '7985', '8007', '8023', '8058', '8060', '8076', '8088', '8132', '8175', '8188', '8279', '8293', '8295', '8312', '8320', '8322', '8356', '8372', '8490', '8521', '8528', '8563', '8569', '8605', '8637', '8640', '8692', '8700', '8767', '8770', '8806', '8838', '8908', '8913', '8957', '8977', '9000', '9049', '9055', '9070', '9136', '9154', '9220', '9252', '9286', '9329', '9340', '9349', '9370', '9473', '9474', '9556', '9565', '9593', '9644', '9647', '9655', '9667', '9704', '9822', '9845', '9872', '9958', '9962', '9966', '10006', '10042', '10088', '10096', '10154', '10172', '10179', '10242', '10373', '10531', '10669', '10675', '10745', '10784', '10800', '10829', '10837', '10895', '10909', '10937', '10979', '11010', '11036', '11051', '11093', '11112', '11136', '11177', '11183', '11220', '11233', '11260', '11283', '11301', '11320', '11359', '11413', '11441', '11454', '11492', '11516', '11524', '11572', '11594', '11662', '11694', '11723', '11776', '11799', '11841', '11864', '11893', '11917', '11922', '11927', '11982', '11983', '11992', '12011', '12025', '12039', '12042', '12079', '12192', '12281', '12284', '12287', '12310', '12411', '12428', '12437', '12448', '12510', '12547', '12551', '12555', '12572', '12579', '12619', '12645', '12672', '12679', '12707', '12729', '12760', '12761', '12770', '12777', '12806', '12869', '12878', '12906', '12912', '12927', '13012', '13030', '13034', '13056', '13144', '13197', '13239', '13249', '13299', '13375', '13395', '13407', '13408', '13574', '13602', '13668', '13681', '13741', '13748', '13749', '13798', '13861', '13878', '13917', '14113', '14138', '14147', '14181', '14184', '14193', '14263', '14301', '14304', '14316', '14406', '14455', '14457', '14490', '14567', '14621', '14630', '14640', '14649', '14681', '14695', '14697', '14732', '14741', '14856', '14858', '14889', '14910', '14940', '14942', '14944', '15023', '15107', '15233', '15293', '15327', '15366', '15380', '15418', '15420', '15455', '15481', '15516', '15566', '15576', '15597', '15619', '15635', '15660', '15666', '15695', '15768', '15774', '15779', '15781', '15816', '15837', '15877', '15899', '15955', '16020', '16075', '16134', '16151', '16168', '16197', '16201', '16206', '16207', '16277', '16284', '16301', '16310', '16393', '16418', '16426', '16427', '16439', '16468', '16606', '16612', '16649', '16689', '16833', '16843', '16850', '16875', '16936', '16958', '17012', '17038', '17074', '17082', '17102', '17106', '17152', '17158', '17182', '17185', '17233', '17314', '17324', '17357', '17400', '17403', '17480', '17506', '17522', '17571', '17664', '17669', '17674', '17729', '17758', '17760', '17797', '17824', '17833', '17866', '17879', '17911', '17933', '18011', '18058', '18094', '18109', '18120', '18127', '18206', '18210', '18342', '18423', '18442', '18445', '18467', '18471', '18483', '18522', '18582', '18643', '18661', '18721', '18744', '18754', '18780', '18809', '18825', '18834', '18863', '18889', '18905', '18940', '18966', '19150', '19164', '19168', '19189', '19235', '19276', '19291', '19327', '19366', '19405', '19410', '19521', '19571', '19579', '19617', '19708', '19722', '19738', '19802', '19811', '19857', '19989', '20014', '20026', '20081', '20106', '20107', '20184', '20214', '20221', '20236', '20282', '20341', '20382', '20390', '20401', '20405', '20488', '20493', '20506', '20542', '20562', '20578', '20641', '20648', '20701', '20762', '20826', '20953', '20964', '21001', '21093', '21117', '21206', '21350', '21401', '21405', '21456', '21465', '21490', '21510', '21564', '21570', '21639', '21655', '21659', '21679', '21694', '21728', '21734', '21749', '21880', '21906', '21988', '22004', '22032', '22036', '22042', '22085', '22098', '22127', '22225', '22236', '22237', '22295', '22314', '22324', '22353', '22365', '22407', '22435', '22521', '22538', '22568', '22579', '22646', '22683', '22709', '22756', '22774', '22780', '22830', '22848', '22849', '22883', '22898', '22901', '22923', '22966', '22976', '23020', '23028', '23058', '23171', '23182', '23186', '23335', '23336', '23350', '23382', '23406', '23443', '23470', '23488']
#711
##715 unstable pts.


Streaming output truncated to the last 5000 lines.
---------------14523---------------
x_occurrenceid=14528
---------------14528---------------
x_occurrenceid=14529
---------------14529---------------
x_occurrenceid=14542
---------------14542---------------
x_occurrenceid=14548
---------------14548---------------
x_occurrenceid=14551
---------------14551---------------
x_occurrenceid=14554
---------------14554---------------
x_occurrenceid=14557
---------------14557---------------
x_occurrenceid=14558
---------------14558---------------
x_occurrenceid=14566
---------------14566---------------
x_occurrenceid=14567
---------------14567---------------
---------------UNSTABLE-14567---------------
x_occurrenceid=14572
---------------14572---------------
x_occurrenceid=14573
---------------14573---------------
x_occurrenceid=14574
---------------14574---------------
x_occurrenceid=14577
---------------14577---------------
x_occurrenceid=14578
---------------14578---------------
x_occurrencei

In [126]:
print(a_results)
print(len(a_results))

['182', '188', '189', '190', '240', '278', '279', '285', '299', '302', '332', '366', '375', '407', '411', '412', '439', '442', '503', '569', '625', '654', '700', '707', '800', '813', '859', '877', '903', '919', '943', '966', '1066', '1071', '1115', '1121', '1133', '1140', '1246', '1303', '1327', '1336', '1342', '1353', '1367', '1372', '1411', '1462', '1548', '1568', '1619', '1661', '1677', '1679', '1704', '1729', '1738', '1742', '1756', '1789', '1825', '1860', '1886', '2041', '2125', '2142', '2146', '2148', '2167', '2201', '2264', '2358', '2457', '2482', '2584', '2598', '2600', '2750', '2755', '2762', '2774', '2815', '2831', '2859', '2863', '2905', '2954', '2980', '3010', '3076', '3111', '3124', '3170', '3180', '3260', '3325', '3326', '3334', '3441', '3464', '3586', '3591', '3611', '3617', '3630', '3635', '3784', '3821', '3840', '3888', '3892', '3909', '3910', '3943', '3952', '3973', '3974', '4001', '4038', '4067', '4068', '4194', '4216', '4221', '4332', '4338', '4357', '4390', '4405',

In [65]:
#rslt_df_3d_fi02_idata = rslt_df_3d_fi02_idata.set_index("measurement_datetime")